In [1]:
import dask.dataframe as dd
from dask.dataframe.utils import make_meta
from neo4j import GraphDatabase
from dask.distributed import Client, LocalCluster, get_worker
import dask
import dill
import os

In [2]:
def addActivity(tx, activityName):
    return tx.run("MERGE (p:Activity {name: $activityName})", activityName=activityName)

def setLinks(row):
    row['predecessor'] = row['activityNameEN'].shift(1); #lag(1)
    row['successor'] = row['activityNameEN'].shift(-1); #lead(1)
    return row;

def getNeo4jCredentials():
    return {
        "host": "bolt://34.239.227.164:7687",
        "user": "neo4j",
        "password": "members-goals-technique"
    }

In [11]:
columnTypes = {
    'case:IDofConceptCase': 'string',
    'case:Includes_subCases': 'string',
    'case:Responsible_actor': 'string',
    'case:caseProcedure': 'string',
    'dateStop': 'string'
}
df = dd.read_csv('BPIC15_1.csv', dtype=columnTypes)

In [4]:
client = Client(n_workers=1)
client

2022-12-04 12:35:46,767 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-3jwvw5_9', purging
2022-12-04 12:35:46,767 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-dj1i26g1', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 5,Total memory: 7.67 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45747,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 5
Started: Just now,Total memory: 7.67 GiB
Comm: tcp://127.0.0.1:38467,Total threads: 5
Dashboard: http://127.0.0.1:37409/status,Memory: 7.67 GiB
Nanny: tcp://127.0.0.1:37849,


In [5]:
creds = getNeo4jCredentials()

In [12]:
def experiment(result):
    driver = GraphDatabase.driver(creds.get('host'), auth=(creds.get('user'), creds.get('password')))
#     with driver.session() as session:
#         result.apply(lambda activityName: session.run("MERGE (p:Activity {name: $activityName})", activityName=activityName),
#                      meta=('activityNameEN', 'object')).compute()
#     session.close()
#     driver.close()
    return result

In [13]:
df['successor'] = ''
df['predecessor'] = ''
df = df.sort_values(by='time:timestamp').groupby('case:concept:name').apply(setLinks, meta=df)

In [14]:
result = df['activityNameEN'].unique()

In [15]:
f_result = client.scatter(result, broadcast=True)

In [16]:
output = client.submit(experiment, f_result)

In [17]:
x = output.result()

In [30]:
def test(activityName, session):
    d_session = dill.loads(session)
    d_session.run("MERGE (p:Activity {name: $activityName})", activityName=activityName)
    return activityName

In [19]:
driver = GraphDatabase.driver(creds.get('host'), auth=(creds.get('user'), creds.get('password')))
with driver.session() as session:
    x.apply(test, meta=('activityNameEN', 'object'), session=dill.dumps(session)).compute()
    session.close()
driver.close()

In [28]:
task = x.apply(test, meta=('activityNameEN', 'object'))

In [29]:
task.compute()

0                   register submission date request
1                         phase application received
2              enter senddate procedure confirmation
3      registrer date of publishing received request
4                     enter senddate acknowledgement
                           ...                      
284                        phase draft decision sent
285                         appealed to higher court
286            create letter requesting updated plan
287                     phase updated plan requested
288                      phase updated plan received
Name: activityNameEN, Length: 289, dtype: object

In [ ]:
# driver = GraphDatabase.driver(creds.get('host'), auth=(creds.get('user'), creds.get('password')))

# dill_file = os.path.join(os.getcwd(), 'dbconn.dill')
# with open(dill_file, 'wb') as f:
#     dill.dump(driver, f)

# scattered_conn = client.scatter(dill.dumps(driver), broadcast=True)

# scattered_file = client.scatter(dill_file, broadcast=True)

# with open(dill_file, 'rb') as f:
#     result = dill.load(f)

# def connect_worker_db():
#     connection = dill.load(open(os.path.join(os.getcwd(), 'dbconn.dill'), 'rb'))
#     worker = get_worker()
#     worker.driver = connection